<a href="https://colab.research.google.com/github/filipefbn/jupyter_intro_class_ibd/blob/master/notebooks/Exerc%C3%ADcios%20parte%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import io
import sqlite3
import pandas as pd

## Exercícios

Neste exercício, vamos trabalhar com dados de uma empresa fictícia de consultoria denominada **IBD**.

**Considere o seguinte esquema relacional:**

![alt text](https://raw.githubusercontent.com/filipefbn/jupyter_intro_class_ibd/master/data/schemaexercicio.png)

Nenhum atributo admite valor nulo.

Apenas as seguintes chaves primárias são numéricas: `Cliente.CodCli`, `Consultor.CPF`


### Download dos dados da empresa

In [2]:
!wget --quiet -P /tmp/ https://raw.githubusercontent.com/filipefbn/jupyter_intro_class_ibd/master/data/exercicio.sql

### Carregando os dados

In [3]:
conn = sqlite3.connect('/tmp/consult.db')
cursor = conn.cursor()

In [4]:
f = io.open('/tmp/exercicio.sql', 'r', encoding='utf-8')
sql = f.read()
cursor.executescript(sql)

In [5]:
cliente = pd.read_sql_query("SELECT * FROM Cliente", conn)
projeto = pd.read_sql_query("SELECT * FROM Projeto", conn)
consultor = pd.read_sql_query("SELECT * FROM Consultor", conn)
equipe = pd.read_sql_query("SELECT * FROM Equipe", conn)
viagem = pd.read_sql_query("SELECT * FROM Viagem", conn)
local = pd.read_sql_query("SELECT * FROM Local", conn)

## Consultas

### 1. Que consultores (CPF, nome e estado de nascimento) não nasceram em São Paulo (SP)?


In [130]:
df = pd.read_sql_query("SELECT C.CPF, C.NomeCon, C.UFNascCon"
                       + " FROM Consultor AS C"
                       + " WHERE C.UFNASCCON <> 'SP'", conn)
df

,CPF,NOMECON,UFNASCCON
0,13414263351,Samuel Araujo Azevedo,PR
1,71960563530,Diogo Cavalcanti Rocha,PB
2,3074973994,Julieta Maciel Sousa,RJ
3,29762035631,Caio Dias Ribeiro,MG
4,17787063122,Larissa Fernandes Araujo,MG
5,31266012516,Leonor Goncalves Rocha,MG
6,85726311353,Brenda Oliveira Goncalves,TO


### 2. Que projetos (sigla e nome) não são do cliente "Facebook"?


In [7]:
df = pd.read_sql_query("SELECT P.Sigla, P.NomePro"
                       + " FROM Projeto AS P"
                       + " WHERE P.CodCli <> (SELECT C.CodCli"
                                             + " FROM Cliente AS C"
                                             + " WHERE C.NomeCli = 'Facebook')"
                       , conn)
df

,SIGLA,NOMEPRO
0,NMT,Neural Machine Translation (Chinês para Portug...
1,Search,Otimização Sistema de Busca
2,Spam,Detecção de Spam (Gmail)
3,Fake,Detecção de Fake News
4,SDC,Self Driving Car
5,Pay,Sistema de Pagamento
6,RecSys,Sistema de Recomendação
7,Fraud,Detecção de Fraude


### 3. Para que UFs viajaram os consultores que participam do projeto de sigla "RecSys"?

In [8]:
df = pd.read_sql_query("SELECT UFLoc"
                      + " FROM Local L"
                      + " INNER JOIN Viagem V"
                      + " ON L.NomeLoc = V.Destino"
                      + " WHERE V.Sigla = 'RecSys'"
                      + " GROUP BY UFLoc"
                      , conn)
df

,UFLOC
0,MG
1,SP


### 4. Que projetos (sigla e nome) não têm equipe?

In [9]:
df = pd.read_sql_query("SELECT P.Sigla, P.NomePro"
                       + " FROM Projeto P"
                       + " LEFT JOIN Equipe E"
                       + " ON P.Sigla = E.Sigla"
                       + " WHERE E.Sigla is null"
                       , conn)
df

,SIGLA,NOMEPRO
0,Search,Otimização Sistema de Busca
1,Fake,Detecção de Fake News
2,SDC,Self Driving Car


### 5. Que projetos (sigla e nome) não têm em sua equipe consultores nascidos antes de 1987?

In [128]:
df = pd.read_sql_query("SELECT P.Sigla, P.NomePro"
                       + " FROM Projeto P"
                       + " EXCEPT"
                       + " SELECT Pj.Sigla, Pj.NomePro"
                       + " FROM Projeto Pj"
                       + " INNER NATURAL JOIN Equipe"
                       + " INNER NATURAL JOIN Consultor C"
                       + " WHERE C.DataNascCon < '1987'"
                      
                       , conn)
df

,SIGLA,NOMEPRO
0,Fake,Detecção de Fake News
1,Fraud,Detecção de Fraude
2,NMT,Neural Machine Translation (Chinês para Portug...
3,SDC,Self Driving Car
4,Search,Otimização Sistema de Busca
5,Spam,Detecção de Spam (Gmail)
6,VR,Hardware para Oculus VR


### 6. Que consultores (CPF e nome) viajaram por seu estado natal (origem ou destino)?


In [48]:
df = pd.read_sql_query("SELECT DISTINCT CPF, NomeCon"
                       + " FROM Local AS L INNER JOIN (Viagem AS V INNER NATURAL JOIN Consultor AS C)"
                       + " ON L.NomeLoc = V.Origem OR L.NomeLoc = V.Destino"
                       + " WHERE L.UFLoc = C.UFNascCon"
                       , conn)
df

,CPF,NomeCon
0,44434353608,Melissa Carvalho Cunha
1,29762035631,Caio Dias Ribeiro
2,34646193892,Rodrigo Araujo Cardoso
3,17787063122,Larissa Fernandes Araujo
4,62465487841,Lucas Fernandes Costa


### 7. Para que locais não houve nenhuma viagem?

In [62]:
df = pd.read_sql_query("SELECT L.NomeLoc"
                       + " FROM Local L"
                       + " LEFT JOIN Viagem V"
                       + " ON L.NomeLoc = V.Destino"
                       + " WHERE V.Destino is null"
                       , conn)
df

,NOMELOC
0,Amazon
1,IBD-PR
2,Netflix
3,OpenAI
4,Spotify


### 8. Em que projetos (sigla e nome) foram realizadas viagens para a California (CA) desde o início de 2019?
OBS: Como o banco foi populado usando a função [`datetime`](https://www.sqlite.org/lang_datefunc.html) do SQLite para as datas, podemos usar DataViagem > '2019' com o `WHERE` (ver link para mais informações).

In [86]:
df = pd.read_sql_query("SELECT DISTINCT P.Sigla, P.NomePro FROM Projeto P" 
                       + " INNER NATURAL JOIN (Viagem V"
                       + " INNER JOIN Local L"
                       + " ON V.Destino = L.NomeLoc)"
                       + " WHERE L.UFLoc = 'CA'"
                       + " AND V.DataViagem > '2019'"
                       , conn)
df

,P.Sigla,P.NomePro
0,WhatsApp,Cliente desktop para o WhatsApp
1,VR,Hardware para Oculus VR


### 9. Que consultores (CPF e nome) viajaram por um projeto de cuja equipe não fazem parte?
OBS: Apesar de não intuitivo, isso é possível, pois no nosso schema temos que Viagem.CPF 🡪 Consultor.CPF e Viagem.Sigla 🡪 Projeto.Sigla.

In [152]:
df = pd.read_sql_query("SELECT R.CPF, R.NomeCon"
                       + " FROM (SELECT V.CPF, C.NomeCon, V.Sigla"
                       + " FROM Viagem V"
                       + " INNER NATURAL JOIN Consultor C"
                       + " EXCEPT"
                       + " SELECT CPF, NomeCon, Sigla"
                       + " FROM Consultor"
                       + " INNER NATURAL JOIN Equipe) AS R"
                       
                      , conn)
df

,CPF,NomeCon
0,34646193892,Rodrigo Araujo Cardoso
1,44434353608,Melissa Carvalho Cunha


In [131]:
viagem

,SIGLA,CPF,DATAVIAGEM,ORIGEM,DESTINO
0,NMT,44434353608,2019-08-11 22:02:41,IBD-SP,Baidu
1,Spam,71960563530,2019-08-14 22:02:41,Google,IBD-MG
2,RecSys,29762035631,2019-08-18 22:02:41,IBD-MG,iFood
3,VR,34646193892,2019-08-22 22:02:41,OpenAI,IBD-SP
4,Pay,17787063122,2019-08-22 22:02:41,iFood,IBD-SP
5,WhatsApp,31266012516,2019-08-24 22:02:41,IBD-SP,Facebook
6,NMT,44434353608,2019-08-29 22:02:41,Baidu,IBD-SP
7,NMT,34646193892,2019-08-29 22:02:41,Baidu,IBD-SP
8,Fraud,71960563530,2019-08-31 22:02:41,IBD-SP,Nubank
9,Spam,44434353608,2019-09-05 22:02:41,IBD-PR,Google


In [133]:
consultor

,CPF,NOMECON,DATANASCCON,UFNASCCON
0,13414263351,Samuel Araujo Azevedo,1990-09-25 22:02:39,PR
1,71960563530,Diogo Cavalcanti Rocha,1991-09-25 22:02:39,PB
2,44434353608,Melissa Carvalho Cunha,1994-09-25 22:02:39,SP
3,62465487841,Lucas Fernandes Costa,1998-09-25 22:02:39,SP
4,3074973994,Julieta Maciel Sousa,1984-09-25 22:02:39,RJ
5,29762035631,Caio Dias Ribeiro,1997-09-25 22:02:39,MG
6,34646193892,Rodrigo Araujo Cardoso,2000-09-25 22:02:39,SP
7,17787063122,Larissa Fernandes Araujo,1977-09-25 22:02:39,MG
8,31266012516,Leonor Goncalves Rocha,1979-09-25 22:02:39,MG
9,85726311353,Brenda Oliveira Goncalves,1981-09-25 22:02:39,TO


In [132]:
equipe

,SIGLA,CPF
0,NMT,44434353608
1,Fraud,44434353608
2,Fraud,71960563530
3,Spam,44434353608
4,Spam,71960563530
5,RecSys,29762035631
6,RecSys,3074973994
7,RecSys,62465487841
8,WhatsApp,17787063122
9,WhatsApp,31266012516


### 10. Que consultores (CPF e nome) não estão em nenhuma equipe de projeto que tenha iniciado em 2019?


In [118]:
df = pd.read_sql_query("SELECT C.CPF, C.NomeCon"
                       + " FROM Consultor C"
                       + " WHERE C.CPF EXCEPT"
                       + " SELECT DISTINCT CPF, NomeCon"
                       + " FROM Consultor"
                       + " INNER NATURAL JOIN Equipe E"
                       + " INNER NATURAL JOIN Projeto P"
                       + " WHERE P.DataInicio >= 2019 AND P.DataInicio < 2020"
                       
                       , conn)
df

,CPF,NOMECON
0,13414263351,Samuel Araujo Azevedo
1,71960563530,Diogo Cavalcanti Rocha
2,85726311353,Brenda Oliveira Goncalves
